In [7]:
import numpy as np
from scipy.integrate import dblquad
from scipy.optimize import fsolve
import sympy as sp
import matplotlib.pyplot as plt

In [8]:
# 为避免混淆，用x,y表示sympy变量，x1,x2,x3表示实际坐标
x, y = sp.symbols('x y')
a, b, c, d, e, f = sp.symbols('a b c d e f')
func0 = a * x * y + b * x ** 2 + c * y ** 2 + d * x + e * y + f  # 曲面函数可修改

def metric(x1, x2, x3):
    return np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])  # 度规可修改

def area_element(x1, x2, func1):
    dfdx = sp.diff(func1, x)
    dfdy = sp.diff(func1, y)
    x3 = func1.evalf(subs={sp.symbols('x'): x1, sp.symbols('y'): x2})
    dzdx = dfdx.evalf(subs={sp.symbols('x'): x1, sp.symbols('y'): x2})
    dzdy = dfdy.evalf(subs={sp.symbols('x'): x1, sp.symbols('y'): x2})
    g = metric(x1, x2, x3)
    return sp.sqrt((g[0][0]+g[2][2]*dzdx**2)*(g[1][1]+g[2][2]*dzdy**2)-(g[0][1]+g[2][2]*dzdx*dzdy)**2)

def boundary(x1, x2):
    return x1 ** 2 + x2 ** 2 - 1  # 边界条件可修改，这里是单位圆

def y_bounds(x1):
    return [-np.sqrt(1 - x1 ** 2), np.sqrt(1 - x1 ** 2)]  # 这里要跟上面的边界条件一起修改

points = [(1, 0, 1), (0, 1, 2), (-1, 0, 3), (0, -1, 4)]  # 边界上的四个点，要与上面的边界条件一同修改

In [9]:
def calculate_function(AB):  # 给定a,b,根据边界条件解c,d,e,f
    A = np.empty((4, 6))
    z_vector = np.empty((4, 1))

    for i, (x1, x2, x3) in enumerate(points):
        A[i, :] = [x1 * x2, x1 ** 2, x2 ** 2, x1, x2, 1]
        z_vector[i, :] = x3

    ab = np.array([AB[0], AB[1]])

    A_prime = A[:, 2:]
    b = z_vector - A[:, :2] @ ab.reshape(-1, 1)

    solution = np.linalg.solve(A_prime, b)
    c, d, e, f = solution.flatten()

    local_func = func0.evalf(
        subs={sp.symbols('a'): AB[0], sp.symbols('b'): AB[1], sp.symbols('c'): c, sp.symbols('d'): d,
              sp.symbols('e'): e, sp.symbols('f'): f})
    return local_func

def calculate_area(l_fc):
    x_bounds = fsolve(lambda xx: boundary(xx, 0), [-1, 1])  # 和边界条件一起改
    area, error = dblquad(lambda x1, x2: area_element(x1, x2, l_fc), x_bounds[0], x_bounds[1],
                          lambda x1: y_bounds(x1)[0], lambda x1: y_bounds(x1)[1])
    return area

In [10]:
def numerical_gradient(f1, x00, area0, epsilon=1e-5):  # 计算f1位于x00处的数值梯度
    grad = np.zeros_like(x00)
    for i in range(x00.shape[0]):
        x_plus = x00.copy()
        x_plus[i] += epsilon
        grad[i] = (f1(x_plus) - area0) / epsilon
    print('gradient:', grad)
    return grad

def gradient_descent(starting_point, learning_rate, iterations):
    p = np.array(starting_point)
    p = p.astype(np.float64)  # 初始点位于starting point
    x1_list = []  # 点列表
    x2_list = []
    for _ in range(iterations):
        print('step:', _+1)
        print('point:', p)
        x1_list.append(p[0])
        x2_list.append(p[1])
        lc_func = calculate_function(p) # 计算曲面方程
        print('function:', lc_func)
        area = calculate_area(lc_func)
        print('area:', area)
        grad = numerical_gradient(lambda xx: calculate_area(calculate_function(xx)), p, area)  # 计算数值梯度
        p -= learning_rate * grad  # 按梯度移动点p
    return p, x1_list, x2_list

In [ ]:
starting_point = [0, 0]
learning_rate = 0.01
iterations = 1000

(min_point, x1_list, x2_list) = gradient_descent(starting_point, learning_rate, iterations)
plt.plot(x1_list, x2_list)
plt.show()

print("面积极小时曲面:", f'{min_point[0]}*x*y + {min_point[1]}*x**2 + {c}*y**2 + {d}*x + {e}*y + {f}')
print("极小值:", calculate_area(calculate_function(min_point)))
